In [ ]:
# import statements
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
pd.set_option('display.max_columns', None)

def concat_files(base_path):
    dfs = []
    for file in os.listdir(base_path):
        if '样本明细' in file and '~' not in file:
            print(file)
            df = pd.read_excel(base_path + file)
            dfs.append(df)

    all_df = pd.concat(dfs)
    print(len(all_df))
    return all_df

# df = concat_files('./')

def clean_null(df):
    # 找出所有标注结果的列
    cols = [item for item in df.columns if ("标注结果-" in item and '多轮' not in item)]
    print(df[cols].isnull().sum())
    print("Before dropping nulls: ", df.shape)
    
    df = df[~df[cols].isnull().any(axis=1)]
    print("After dropping nulls: ", df.shape)
    
    return df

def get_score(df):
    df = df[df['样本状态'] == '已评测']
    score_dict = {'不切题': 0, '结果不满意': 1, '基本满意': 2, '满意': 3, '超出预期': 3}
    # 筛选cols这个列表中只包含“标注结果-”样式的item
    cols = [item for item in df.columns if ("标注结果-" in item and '多轮' not in item)]
    for col in cols:
        # 新增一列score，将eval_results这一列的值映射到score_dict中
        df.loc[:, col.replace('标注结果-', '分数_')] = df[col].map(score_dict)
        df.loc[:, col.replace('标注结果-', '分数100_')] = df.loc[:, col.replace('标注结果-', '分数_')] / 3 * 100
    print([col for col in df.columns if "分数100_" in col])
    return df

def process_df(df):
    df = df.copy()
    df = clean_null(df)
    df = get_score(df)

    return df

In [45]:
from glob import glob
sample_files = glob('正式评测集/评测结果/任务*.csv')
df_auto = pd.concat([pd.read_csv(f) for f in sample_files])

# 多轮和代码用另外的样本明细
df_auto = df_auto[(~df_auto['一级分类'].str.contains('多轮')) & (~df_auto['二级分类'].str.contains('代码'))]
print(df_auto.shape)

(14064, 61)


In [46]:
df_code1 = pd.read_csv('正式评测集/评测结果/代码1任务18531-样本明细-sample.file.ea2cee150edf6443d5fb8bf61fe24204-2025-05-01_10_45_59.csv')
df_code2 = pd.read_csv('正式评测集/评测结果/代码2任务18540-样本明细-sample.file.450a62dbac22b28859abd8b3ba1402c4-2025-05-01_10_35_12.csv')

# 把code1中包含code2的去掉，取code2的值
df_code = pd.concat([df_code1[~df_code1['Prompt序列号'].isin(df_code2['Prompt序列号'].unique())], df_code2])
print(df_code.shape)
print(df_code['Prompt序列号'].nunique())

(435, 61)
145


In [47]:
df_duolun = pd.read_csv("正式评测集/评测结果/多轮-任务18541-样本明细-sample.file.fa7e10d75bbe8461e816e44f327d3372-2025-05-01_10_44_44.csv")
df_duotu = pd.read_csv("正式评测集/评测结果/多图-任务18530-样本明细-sample.file.8edfffdc19767f13d5e3f7db6c004c66-2025-05-01_10_34_48.csv")

print(df_duolun.shape)
print(df_duolun['Prompt序列号'].nunique())
print(df_duotu.shape)
print(df_duotu['Prompt序列号'].nunique())

(594, 79)
198
(897, 61)
299


In [48]:
df = pd.concat([df_auto, df_code, df_duolun, df_duotu])
print(df.shape)
print(df['Prompt序列号'].nunique())

(15990, 79)
5330


In [49]:
df = process_df(df)

标注结果-多模型打分（5档）-GPT-4.1                             4
标注结果-多模型打分（5档）-qwen-vl-max-latest                  4
标注结果-多模型打分（5档）-step-1o-vision-32k                  4
标注结果-多模型打分（5档）-hunyuan-turbos-vision               4
标注结果-多模型打分（5档）-Gemini-2.5-pro-preview-0325         4
标注结果-多模型打分（5档）-Doubao-1.5-vision-pro-32k-250115    4
dtype: int64
Before dropping nulls:  (15990, 79)
After dropping nulls:  (15986, 79)
['分数100_多模型打分（5档）-GPT-4.1', '分数100_多模型打分（5档）-qwen-vl-max-latest', '分数100_多模型打分（5档）-step-1o-vision-32k', '分数100_多模型打分（5档）-hunyuan-turbos-vision', '分数100_多模型打分（5档）-Gemini-2.5-pro-preview-0325', '分数100_多模型打分（5档）-Doubao-1.5-vision-pro-32k-250115']


/var/folders/lt/47tfwyrj1_543j06tbpq7_q00000gn/T/ipykernel_45767/4032204410.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, col.replace('标注结果-', '分数_')] = df[col].map(score_dict)
/var/folders/lt/47tfwyrj1_543j06tbpq7_q00000gn/T/ipykernel_45767/4032204410.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, col.replace('标注结果-', '分数100_')] = df.loc[:, col.replace('标注结果-', '分数_')] / 3 * 100
/var/folders/lt/47tfwyrj1_543j06tbpq7_q00000gn/T/ipykernel_45767/4032204410.py:41: SettingWithCo

### 算分

In [50]:
import pandas as pd
import re

# 定义提取函数
def extract_info(text):
    # 使用正则表达式提取链接
    link_pattern = r'\!\[.*?\]\((.*?)\)'
    link_match = re.search(link_pattern, text)
    link = link_match.group(1) if link_match else None
    
    # 提取 \n 后面的内容
    content_pattern = r'\n(.*)'
    content_match = re.search(content_pattern, text)
    content = content_match.group(1) if content_match else None
    
    return pd.Series([link, content], index=['原图', '原题'])

In [62]:
def answer_extract(ans, q):
    s = ans.split('A: ')
    if len(s) == 2:
        ans = s[-1]
        # return ans
    else:
        t = q.split('A: ')
        ans = 'A: '.join(s[len(t):])

    # 第三步处理：判断 ans 是否包含 </details>\n\n
    if '</details>\n\n' in ans:
        # 只取 </details>\n\n 往后的内容
        ans = ans.split('</details>\n\n', 1)[-1]

    return ans

df['hunyuan-turbos-vision'] = df.apply(lambda row: answer_extract(row['模型回答-hunyuan-turbos-vision'], row['问题']), axis=1)

In [63]:
score_columns = df.filter(like='分数100_').columns
renamed_cols = [item.replace("多模型打分（5档）-", "") for item in score_columns]
df.rename(columns=dict(zip(score_columns, renamed_cols)), inplace=True)
score_columns = df.filter(like='分数100_').columns

agg_prompt_dict = {col:'mean' for col in score_columns}
df_agg = df.groupby(['一级分类','二级分类','三级分类', 'Prompt序列号','问题','参考答案','hunyuan-turbos-vision'], dropna=False).agg(agg_prompt_dict).reset_index()
# 应用提取函数
# df_remove_agg[['图片', '题目']] = df_remove_agg['问题'].apply(extract_info)
df_agg[['图片', '题目']] = df_agg['问题'].apply(extract_info)
print(df_agg.shape)
df_agg.head(1)

(5329, 15)


,一级分类,二级分类,三级分类,Prompt序列号,问题,参考答案,hunyuan-turbos-vision,分数100_GPT-4.1,分数100_qwen-vl-max-latest,分数100_step-1o-vision-32k,分数100_hunyuan-turbos-vision,分数100_Gemini-2.5-pro-preview-0325,分数100_Doubao-1.5-vision-pro-32k-250115,图片,题目
0,图生文OCR,自然场景OCR,发票与小票,TPROMPT047c1e316105,第1轮: ![loading](https://hunyuan-eval-125834470...,688元,The total amount paid according to the receipt...,55.555556,100.0,55.555556,77.777778,77.777778,33.333333,https://hunyuan-eval-1258344703.cos.ap-guangzh...,How much did you pay in total in the picture?


In [64]:
df_agg.to_excel("4月问题明细-包含混元回答.xlsx",index=False)

In [44]:
df_agg.groupby(['一级分类']).count().reset_index()

,一级分类,二级分类,三级分类,Prompt序列号,问题,参考答案,分数100_Doubao-1.5-vision-pro-32k-250115,分数100_GPT-4.1,分数100_qwen-vl-max-latest,分数100_step-1o-vision-32k,分数100_hunyuan-turbos-vision,分数100_Gemini-2.5-pro-preview-0325,图片,题目
0,图生文OCR,343,343,343,343,343,343,343,343,343,343,343,343,343
1,图生文图像内容创作,336,336,336,336,336,336,336,336,336,336,336,336,336
2,图生文图像分析推理,1325,1325,1325,1325,1325,1325,1325,1325,1325,1325,1325,1325,1325
3,图生文图像基础识别,1043,1043,1043,1043,1043,1043,1043,1043,1043,1043,1043,1043,1043
4,图生文图像知识问答,685,685,685,685,685,685,685,685,685,685,685,685,685
5,图生文多轮对话,198,198,198,198,198,198,198,198,198,198,198,198,198
6,图生文视觉数学,731,731,731,731,731,731,731,731,731,731,731,731,731
7,多图理解与推理,299,299,299,299,299,299,299,299,299,299,299,299,299


In [28]:
agg_prompt_dict = {col:'mean' for col in score_columns}

agg_3rd_dict = agg_prompt_dict.copy()
agg_3rd_dict['Prompt序列号'] = 'count'
df_3rd_removed_agg = df_agg.groupby(['一级分类','二级分类','三级分类']).agg(agg_3rd_dict).reset_index()

agg_2nd_dict = agg_prompt_dict.copy()
agg_2nd_dict['Prompt序列号'] = 'count'
df_2nd_removed_agg = df_agg.groupby(['一级分类','二级分类']).agg(agg_2nd_dict).reset_index()

agg_1st_dict = agg_prompt_dict.copy()
agg_1st_dict['Prompt序列号'] = 'sum'
df_1st_removed_agg = df_2nd_removed_agg.groupby(['一级分类']).agg(agg_1st_dict).reset_index()

df_1st_removed_agg

,一级分类,分数100_Doubao-1.5-vision-pro-32k-250115,分数100_GPT-4.1,分数100_qwen-vl-max-latest,分数100_step-1o-vision-32k,分数100_hunyuan-turbos-vision,分数100_Gemini-2.5-pro-preview-0325,Prompt序列号
0,图生文OCR,69.346269,67.650343,71.914029,72.324825,67.306730,76.288981,343
1,图生文图像内容创作,77.754346,77.317082,69.316327,73.009826,75.912698,69.260393,594
2,图生文图像分析推理,72.836093,77.482836,74.744084,70.911429,76.755373,83.166273,1372
3,图生文图像基础识别,76.459926,78.874628,73.516338,75.615164,75.059478,83.044567,1107
4,图生文图像知识问答,78.690162,70.809233,74.732874,73.156788,75.043266,83.808816,685
5,图生文多轮对话,75.645342,75.897868,75.196409,70.538721,70.173962,78.563412,198
6,图生文视觉数学,72.603550,61.420827,61.760041,53.732526,61.523828,80.209403,731
7,多图理解与推理,77.011721,78.645559,75.637282,75.571970,71.806307,77.800319,299


In [29]:
output_file = './正式评测集/评测结果/4月大版本结果.xlsx'

with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    # Write each DataFrame to a different worksheet
    df_1st_removed_agg.sort_values('一级分类').to_excel(writer, sheet_name='一级汇总', index=False)
    df_2nd_removed_agg.sort_values(['一级分类','二级分类']).to_excel(writer, sheet_name='二级汇总', index=False)
    df_3rd_removed_agg.sort_values(['一级分类','二级分类', '三级分类']).to_excel(writer, sheet_name='三级汇总', index=False)
    df_agg.sort_values(['一级分类','二级分类']).to_excel(writer, sheet_name='问题明细', index=False)

    # Close the Pandas Excel writer and output the Excel file
    writer.save()

/var/folders/lt/47tfwyrj1_543j06tbpq7_q00000gn/T/ipykernel_45767/3398749775.py:11: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
/opt/homebrew/lib/python3.10/site-packages/xlsxwriter/workbook.py:369: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


### 多轮分析

In [65]:
df_duolun = pd.read_csv("正式评测集/评测结果/多轮-任务18541-样本明细-sample.file.fa7e10d75bbe8461e816e44f327d3372-2025-05-01_10_44_44.csv")

print(df_duolun.shape)
print(df_duolun['Prompt序列号'].nunique())

(594, 79)
198


In [66]:
df_duolun = process_df(df_duolun)
df_duolun.head(1)

标注结果-多模型打分（5档）-step-1o-vision-32k                  2
标注结果-多模型打分（5档）-hunyuan-turbos-vision               2
标注结果-多模型打分（5档）-Gemini-2.5-pro-preview-0325         2
标注结果-多模型打分（5档）-Doubao-1.5-vision-pro-32k-250115    2
标注结果-多模型打分（5档）-GPT-4.1                             2
标注结果-多模型打分（5档）-qwen-vl-max-latest                  2
dtype: int64
Before dropping nulls:  (594, 79)
After dropping nulls:  (592, 79)
['分数100_多模型打分（5档）-step-1o-vision-32k', '分数100_多模型打分（5档）-hunyuan-turbos-vision', '分数100_多模型打分（5档）-Gemini-2.5-pro-preview-0325', '分数100_多模型打分（5档）-Doubao-1.5-vision-pro-32k-250115', '分数100_多模型打分（5档）-GPT-4.1', '分数100_多模型打分（5档）-qwen-vl-max-latest']


,任务 ID,ID,提示词 ID,一级分类,二级分类,三级分类,翻译,答案是否可接受,备注,评测人,样本状态,tag,是否overlap,评测用时,更新时间,Prompt序列号,TraceId,问题,参考答案,模型回答-step-1o-vision-32k,模型回答-step-1o-vision-32k-请求详情,模型回答-hunyuan-turbos-vision,模型回答-hunyuan-turbos-vision-请求详情,模型回答-Gemini-2.5-pro-preview-0325,模型回答-Gemini-2.5-pro-preview-0325-请求详情,模型回答-Doubao-1.5-vision-pro-32k-250115,模型回答-Doubao-1.5-vision-pro-32k-250115-请求详情,模型回答-GPT-4.1,模型回答-GPT-4.1-请求详情,模型回答-qwen-vl-max-latest,模型回答-qwen-vl-max-latest-请求详情,标注结果-多模型打分（5档）-step-1o-vision-32k,维度归因-多模型打分（5档）-step-1o-vision-32k,标注结果-多模型打分（5档）-hunyuan-turbos-vision,维度归因-多模型打分（5档）-hunyuan-turbos-vision,标注结果-多模型打分（5档）-Gemini-2.5-pro-preview-0325,维度归因-多模型打分（5档）-Gemini-2.5-pro-preview-0325,标注结果-多模型打分（5档）-Doubao-1.5-vision-pro-32k-250115,维度归因-多模型打分（5档）-Doubao-1.5-vision-pro-32k-250115,标注结果-多模型打分（5档）-GPT-4.1,维度归因-多模型打分（5档）-GPT-4.1,标注结果-多模型打分（5档）-qwen-vl-max-latest,维度归因-多模型打分（5档）-qwen-vl-max-latest,标注结果备注-step-1o-vision-32k,标注结果备注-hunyuan-turbos-vision,标注结果备注-Gemini-2.5-pro-preview-0325,标注结果备注-Doubao-1.5-vision-pro-32k-250115,标注结果备注-GPT-4.1,标注结果备注-qwen-vl-max-latest,反馈-step-1o-vision-32k,反馈-hunyuan-turbos-vision,反馈-Gemini-2.5-pro-preview-0325,反馈-Doubao-1.5-vision-pro-32k-250115,反馈-GPT-4.1,反馈-qwen-vl-max-latest,多轮标注结果-多模型打分（5档）-step-1o-vision-32k,多轮标注结果-多模型打分（5档）-hunyuan-turbos-vision,多轮标注结果-多模型打分（5档）-Gemini-2.5-pro-preview-0325,多轮标注结果-多模型打分（5档）-Doubao-1.5-vision-pro-32k-250115,多轮标注结果-多模型打分（5档）-GPT-4.1,多轮标注结果-多模型打分（5档）-qwen-vl-max-latest,多轮标注结果备注-step-1o-vision-32k,多轮标注结果备注-hunyuan-turbos-vision,多轮标注结果备注-Gemini-2.5-pro-preview-0325,多轮标注结果备注-Doubao-1.5-vision-pro-32k-250115,多轮标注结果备注-GPT-4.1,多轮标注结果备注-qwen-vl-max-latest,归因-step-1o-vision-32k,归因-hunyuan-turbos-vision,归因-Gemini-2.5-pro-preview-0325,归因-Doubao-1.5-vision-pro-32k-250115,归因-GPT-4.1,归因-qwen-vl-max-latest,多轮归因-step-1o-vision-32k,多轮归因-hunyuan-turbos-vision,多轮归因-Gemini-2.5-pro-preview-0325,多轮归因-Doubao-1.5-vision-pro-32k-250115,多轮归因-GPT-4.1,多轮归因-qwen-vl-max-latest,分数_多模型打分（5档）-step-1o-vision-32k,分数100_多模型打分（5档）-step-1o-vision-32k,分数_多模型打分（5档）-hunyuan-turbos-vision,分数100_多模型打分（5档）-hunyuan-turbos-vision,分数_多模型打分（5档）-Gemini-2.5-pro-preview-0325,分数100_多模型打分（5档）-Gemini-2.5-pro-preview-0325,分数_多模型打分（5档）-Doubao-1.5-vision-pro-32k-250115,分数100_多模型打分（5档）-Doubao-1.5-vision-pro-32k-250115,分数_多模型打分（5档）-GPT-4.1,分数100_多模型打分（5档）-GPT-4.1,分数_多模型打分（5档）-qwen-vl-max-latest,分数100_多模型打分（5档）-qwen-vl-max-latest
0,18541,26725685,43828012,图生文多轮对话,单图多轮,长期记忆,NaN,NaN,NaN,李浩浩,已评测,NaN,否,00:33:45,2025-04-29 14:19:26,TPROMPTe8cd6377a87d,NaN,第1轮: ![loading](https://hunyuan-eval-125834470...,第1轮\nQ: 识别图中绿色牌子上的文字\nA: 绿色牌子上的文字为 “HONGSHAN F...,第1轮\nQ: 识别图中绿色牌子上的文字\nA: 绿色牌子上的文字是：\n\n```\n红山...,: null;: null;: null;: null;: null;,第1轮\nQ: 识别图中绿色牌子上的文字\nA: 图中绿色牌子上的文字是：HONGSHAN ...,: null;: null;: null;: null;: null;,第1轮\nQ: 识别图中绿色牌子上的文字\nA: 图中绿色牌子上的文字是：\n\n**HON...,: null;: null;: null;: null;: null;,第1轮\nQ: 识别图中绿色牌子上的文字\nA: 图中绿色牌子上的文字为：\n\nHONGS...,: null;: null;: null;: null;: null;,第1轮\nQ: 识别图中绿色牌子上的文字\nA: 图中绿色牌子上的文字是：\n\n**HON...,: null;: null;: null;: null;: null;,第1轮\nQ: 识别图中绿色牌子上的文字\nA: 图中绿色牌子上的文字是：\n\n```\n...,: null;: null;: null;: null;: null;,不切题,NaN,结果不满意,NaN,不切题,NaN,满意,NaN,满意,NaN,基本满意,NaN,第3轮，拒答,第1轮，文字有漏，不满意\n第3轮，时间错误，不满意,第3轮，拒答,NaN,NaN,第3轮，不满意，时间错误,NaN,NaN,NaN,NaN,NaN,NaN,第1轮: 满意\n\n第2轮: 满意\n\n第3轮: 不切题\n\n第4轮: 满意\n\n第...,第1轮: 结果不满意\n\n第2轮: 满意\n\n第3轮: 结果不满意\n\n第4轮: 满意...,第1轮: 满意\n\n第2轮: 满意\n\n第3轮: 不切题\n\n第4轮: 满意\n\n第...,第1轮: 满意\n\n第2轮: 满意\n\n第3轮: 满意\n\n第4轮: 满意\n\n第5...,第1轮: 满意\n\n第2轮: 满意\n\n第3轮: 满意\n\n第4轮: 满意\n\n第5...,第1轮: 满意\n\n第2轮: 满意\n\n第3轮: 结果不满意\n\n第4轮: 满意\n\...,第1轮: \n\n第2轮: \n\n第3轮: 不合理拒答\n\n第4轮: \n\n第5轮:,第1轮: 文字遗漏\n\n第2轮: \n\n第3轮: 时间错误\n\n第4轮: \n\n第5轮:,第1轮: \n\n第2轮: \n\n第3轮: 不合理拒答\n\n第4轮: \n\n第5轮:,第1轮: \n\n第2轮: \n\n第3轮: \n\n第4轮: \n\n第5轮:,第1轮: \n\n第2轮: \n\n第3轮: \n\n第4轮: \n\n第5轮:,第1轮: \n\n第2轮: \n\n第3轮: 时间错误\n\n第4轮: \n\n第5轮:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,1,33.333333,0,0.0,3,100.0,3,100.0,2,66.666667
